In [1]:
import pandas as pd
customer_data = pd.read_csv("Churn_Modelling.csv")
customer_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
customer_data.drop("RowNumber", inplace=True, axis=1)
customer_data.drop("CustomerId", inplace=True, axis=1)
customer_data.drop("Surname", inplace=True, axis=1)
customer_data.drop("Geography", inplace=True, axis=1)
features = customer_data.drop("Exited", axis=1).copy()
labels = customer_data["Exited"].copy()

In [3]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Gender           10000 non-null  object 
 2   Age              10000 non-null  int64  
 3   Tenure           10000 non-null  int64  
 4   Balance          10000 non-null  float64
 5   NumOfProducts    10000 non-null  int64  
 6   HasCrCard        10000 non-null  int64  
 7   IsActiveMember   10000 non-null  int64  
 8   EstimatedSalary  10000 non-null  float64
 9   Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 781.4+ KB


In [4]:
customer_data.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [11]:
encoded_gender = [1 if i == "Male" else 0 for i in features["Gender"]]
features["Gender"] = encoded_gender

In [12]:
features.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,42,2,0.00,1,1,1,101348.88
1,608,0,41,1,83807.86,1,0,1,112542.58
2,502,0,42,8,159660.80,3,1,0,113931.57
3,699,0,39,1,0.00,2,0,0,93826.63
4,850,0,43,2,125510.82,1,1,1,79084.10


In [13]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Gender           10000 non-null  int64  
 2   Age              10000 non-null  int64  
 3   Tenure           10000 non-null  int64  
 4   Balance          10000 non-null  float64
 5   NumOfProducts    10000 non-null  int64  
 6   HasCrCard        10000 non-null  int64  
 7   IsActiveMember   10000 non-null  int64  
 8   EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 703.2 KB


In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalised_features = scaler.fit_transform(features)

In [21]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.1)
for train_index, test_index in split.split(normalised_features, labels):
    X_train, X_test = normalised_features[train_index], normalised_features[test_index]
    Y_train, Y_test = labels[train_index], labels[test_index]

In [22]:
X_train.shape

(9000, 9)

In [23]:
Y_train.shape

(9000,)

In [24]:
X_test.shape

(1000, 9)

In [25]:
Y_test.shape

(1000,)

In [29]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(max_iter=300)
clf.fit(X_train, Y_train)

MLPClassifier(max_iter=300)

In [30]:
predicted = clf.predict(X_test)

In [31]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, r2_score
confusion_matrix(Y_test, predicted)

array([[763,  33],
       [121,  83]], dtype=int64)

In [32]:
recall_score(Y_test, predicted)

0.4068627450980392

In [33]:
precision_score(Y_test, predicted)

0.7155172413793104

In [34]:
r2_score(Y_test, predicted)

0.05163070253226909